In [8]:
using  Plots
pyplot();

using ForwardDiff, LinearAlgebra

In [5]:
# Line search
function bisection(f,a,b)
    l = 1e-12
    bᵢ = b
    aᵢ = a
    λ = 0
    while bᵢ - aᵢ > l
        λ = (aᵢ + bᵢ)/2
        ∇f= ForwardDiff.derivative(f,λ)
        if abs(∇f) < l # equivalent to ∇f = 0
            return λ
        elseif ∇f > 0  # left move
            aᵢ₊₁ = aᵢ
            bᵢ₊₁ = λ
        else           # move right
            aᵢ₊₁ = λ
            bᵢ₊₁ = bᵢ
        end
        aᵢ = aᵢ₊₁
        bᵢ = bᵢ₊₁
    end
        return λ
end;

function newton(f, x, N = 1000; ϵ = 1e-6, λ = 1.0, a = 0.0, b = 2.0)
    tini = time()                  # Start timer
    ∇(f, x)  = ForwardDiff.gradient(f, x)
    H(f, x) = ForwardDiff.hessian(f, x)

    for i = 1:N
        ∇fᵢ = ∇(f, x)               # Gradient
        if norm(∇fᵢ) < ϵ            # Stopping condition #1
            tend = time() - tini    # Computation time
            return x, f(x), tend, i # Return cost, time, iterations
        end
        d = -H(f, x)\∇fᵢ            # Newton direction
        ls(λ) = f(x + λ*d)          # Line search via bisection
        λ = bisection(ls,a,b)
        x = x + λ*d                 # Move to a new point
    end
    tend = time() - tini            # Computation time
    return x, f(x), tend, N         # Return x, f(x), time, iterations
end;

In [9]:
# We will consider this problem
f(x) = (x[1]-2)^4 + (x[1]-2x[2])^2   # min f(x)
g(x) = x[1]^2 - x[2]                  # g(x) ≦ 0

# Penalty function
B(x) = -1/g(x)

# Initial parameters
μ = 10          # penalty term. Try different values
β = 0.2         # amount of reduction in penalty
ϵ = 1e-7        # tolerance
x₀ = [0.5,1]  # initial feasible point

# Method of multipliers implementation
function barrier_method(μ, f, B, x₀, β, ϵ)
        xᵏ =  x₀
        traj = zeros(2,50)
        traj[:,1] = x₀
        k = 2
        println("\nStarting barrier method...")
        while (abs(μ*B(xᵏ)) > ϵ)
            F(x) = f(x) + μ*B(x)    # barrier problem
            xᵏ,⋅,⋅,⋅ = newton(F,xᵏ) # solver barrier prob. with Newton's
            traj[:,k] = xᵏ          # record trajectory for plotting
            println("Current point: ", xᵏ, "/ Objective : ", f(xᵏ))
            μ = β*μ                # update μ
            k = k+1
        end
        traj = traj[:,1:k-1]
        return traj
end

barrier_method (generic function with 1 method)

In [10]:
traj = barrier_method(μ, f, B, x₀, β, ϵ)


Starting barrier method...
1.3333333333333333
Current point: [0.707944, 1.5315]/ Objective : 8.333201262357004
0.9705779768116337
Current point: [0.796733, 1.19499]/ Objective : 4.634681146202505
1.785070333784634
Current point: [0.862407, 1.03218]/ Objective : 3.1194568484593184
3.466933481265405
Current point: [0.903224, 0.956543]/ Objective : 2.466831025134442
7.1058475981591975
Current point: [0.925305, 0.922163]/ Objective : 2.1785519926323866
15.157676912813807
Current point: [1.9293, 0.964703]/ Objective : 2.499374279955946e-5
-0.36264717024584586
Current point: [1.95941, 0.979713]/ Objective : 2.7159695116509338e-6
-0.3497044206595875
Current point: [1.9765, 0.988253]/ Objective : 3.048831198001497e-7
-0.34266433917389405
Current point: [1.98634, 0.99317]/ Objective : 3.483454452538287e-8
-0.338710847891267
Current point: [1.99204, 0.996019]/ Objective : 4.019693030077436e-9
-0.33645171436763155
Current point: [1.99535, 0.997677]/ Objective : 4.658480177675153e-10


2×12 Array{Float64,2}:
 0.5  0.707944  0.796733  0.862407  0.903224  …  1.98634  1.99204   1.99535 
 1.0  1.5315    1.19499   1.03218   0.956543     0.99317  0.996019  0.997677

In [ ]:
# Plotting the contours of the function optimised and the trajectory
n = 5000
x = range(-200,stop=200,length=n);
y = range(-200,stop=200,length=n);
z = [f([x[i],y[j]]) for j = 1:n, i = 1:n];

# Plot function contours
contour(x,y,z, # Change title accordingly
        levels = [0.25, 1, 2.5 , 5, 9],
        xaxis = ("x₁", (0,2)),
        yaxis = ("x₂", (0,2)),
        clims = (0,12),
        #clabels = true,
        legend = false,
        aspect_ratio = :equal)

# Plot feasible region
plot!(x, x.^2,
        #fill= (10,0.1),
        color = 1)
annotate!([(0.25, 1.75, text("x₁² - x₂ ≤ 0",9,:left,
                RGBA(0.0,0.605603,0.97868,1.0)))])

# Plot trajectory
plot!(traj[1,:], traj[2,:], marker=:o)
annotate!([(0.5, 0.9, text("x₀",9,:left,
                RGBA(0.2422242978521988,0.6432750931576304,0.30444865153411527,1.0) ))])